In [1]:
import pandas as pd
import sqlite3
import json

# LOAD DATA 
orders = pd.read_csv("orders.csv")

with open("users.json", "r") as f:
    users = pd.DataFrame(json.load(f))

conn = sqlite3.connect(":memory:")
cursor = conn.cursor()
with open("restaurants.sql", "r") as f:
    cursor.executescript(f.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

#  MERGE (LEFT JOIN) 
df = (
    orders
    .merge(users, on="user_id", how="left")
    .merge(restaurants, on="restaurant_id", how="left")
)

df["order_date"] = pd.to_datetime(df["order_date"])

#  Q1 
print("\nQ1: Highest average order value cuisine")
print(df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False))

#  Q2 
print("\nQ2: Users spending more than ₹1000")
print((df.groupby("user_id")["total_amount"].sum() > 1000).sum())

#  Q3 
df["rating_range"] = pd.cut(
    df["rating"],
    bins=[0, 3, 4, 5],
    labels=["Low", "Medium", "High"]
)
print("\nQ3: Revenue by rating range")
print(df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False))

#  Q4 
print("\nQ4: Gold members – highest average order value city")
print(
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

#  Q5 
restaurant_count = df.groupby("cuisine")["restaurant_id"].nunique()
revenue = df.groupby("cuisine")["total_amount"].sum()
print("\nQ5: Low restaurant count but high revenue cuisine")
print(pd.concat([restaurant_count, revenue], axis=1)
      .rename(columns={0: "restaurant_count", 1: "revenue"})
      .sort_values(["restaurant_id", "total_amount"], ascending=[True, False]))

#  Q6 
gold_pct = round((df[df["membership"] == "Gold"].shape[0] / df.shape[0]) * 100)
print("\nQ6: % orders by Gold members")
print(gold_pct, "%")

#  Q7 
restaurant_stats = df.groupby("restaurant_name_y").agg(
    avg_order=("total_amount", "mean"),
    orders=("order_id", "count")
)
print("\nQ7: Highest AOV restaurant with <20 orders")
print(restaurant_stats[restaurant_stats["orders"] < 20]
      .sort_values("avg_order", ascending=False)
      .head(1))

#  Q8 
print("\nQ8: Highest revenue Membership + Cuisine")
print(df.groupby(["membership", "cuisine"])["total_amount"]
      .sum()
      .sort_values(ascending=False)
      .head(1))

#  Q9 
df["quarter"] = df["order_date"].dt.to_period("Q")
print("\nQ9: Highest revenue quarter")
print(df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False))


Q1: Highest average order value cuisine
cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

Q2: Users spending more than ₹1000
2544

Q3: Revenue by rating range
rating_range
High      4157357.01
Medium    3599248.98
Low        255018.13
Name: total_amount, dtype: float64

Q4: Gold members – highest average order value city
city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

Q5: Low restaurant count but high revenue cuisine
         restaurant_id  total_amount
cuisine                             
Chinese            120    1930504.65
Italian            126    2024203.80
Indian             126    1971412.58
Mexican            128    2085503.09

Q6: % orders by Gold members
50 %

Q7: Highest AOV restaurant with <20 orders
                     avg_order  orders
restaurant_name_y                     
Restaurant_294     1040.2223

C:\Users\ALOK\AppData\Local\Temp\ipykernel_16632\4221228190.py:25: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


In [2]:
import pandas as pd
import sqlite3
import json

#  Load data
orders = pd.read_csv("orders.csv")

with open("users.json", "r") as f:
    users = pd.DataFrame(json.load(f))

conn = sqlite3.connect(":memory:")
cursor = conn.cursor()
with open("restaurants.sql", "r") as f:
    cursor.executescript(f.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

# - Merge datasets (LEFT JOIN) 
df = (
    orders
    .merge(users, on="user_id", how="left")
    .merge(restaurants, on="restaurant_id", how="left")
)

#  Q1 
q1 = df[df["membership"] == "Gold"].shape[0]
print("1) Total orders by Gold members:", q1)

# Q2
q2 = round(df[df["city"] == "Hyderabad"]["total_amount"].sum())
print("2) Total revenue from Hyderabad:", q2)

# Q3
q3 = df["user_id"].nunique()
print("3) Distinct users who placed at least one order:", q3)

#  Q4 
q4 = round(df[df["membership"] == "Gold"]["total_amount"].mean(), 2)
print("4) Average order value for Gold members:", q4)

#  Q5 
q5 = df[df["rating"] >= 4.5].shape[0]
print("5) Orders placed for restaurants with rating >= 4.5:", q5)

#  Q6 
gold_city_revenue = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

top_gold_city = gold_city_revenue.idxmax()

q6 = df[
    (df["membership"] == "Gold") &
    (df["city"] == top_gold_city)
].shape[0]

print("6) Top revenue city among Gold members:", top_gold_city)
print("   Orders in that city (Gold only):", q6)

1) Total orders by Gold members: 4987
2) Total revenue from Hyderabad: 1889367
3) Distinct users who placed at least one order: 2883
4) Average order value for Gold members: 797.15
5) Orders placed for restaurants with rating >= 4.5: 3374
6) Top revenue city among Gold members: Chennai
   Orders in that city (Gold only): 1337


In [5]:
import pandas as pd
import sqlite3
import json

# Load datasets
orders = pd.read_csv("orders.csv")
print("Orders columns:", orders.columns.tolist())

with open("users.json", "r") as f:
    users = pd.DataFrame(json.load(f))
print("Users columns:", users.columns.tolist())

conn = sqlite3.connect(":memory:")
cursor = conn.cursor()
with open("restaurants.sql", "r") as f:
    cursor.executescript(f.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
print("Restaurants columns:", restaurants.columns.tolist())

#  STEP 2: Perform LEFT JOINS 
final_df = (
    orders
    .merge(users, on="user_id", how="left")
    .merge(restaurants, on="restaurant_id", how="left")
)

#  STEP 3: Proof Outputs 

# Q1: Join key between orders and users
print("\nQ1 Proof - Join key between orders and users:")
print("Used key -> 'user_id'")

# Q2 : Dataset with cuisine and rating
print("\nQ2 Proof - Cuisine & rating source:")
print("Source dataframe -> restaurants")
print("Original format -> SQL (.sql)")

# Q3: Row count preservation due to LEFT JOIN
print("\nQ3 Proof - Row count check:")
print("Orders rows:", orders.shape[0])
print("Final merged rows:", final_df.shape[0])

# Q4: Missing users lead to NaN
print("\nQ4 Proof - Missing user values:")
print(final_df[final_df["name"].isna()].head())

# Q5: Pandas function used for joining
print("\nQ5 Proof - Pandas join function used:")
print("Function -> pandas.merge()")

# Q6: Membership column source
print("\nQ6 Proof - Membership column origin:")
print("membership column found in users.json")

# Q7: Join key with restaurants
print("\nQ7 Proof - Join key with restaurants:")
print("Used key -> 'restaurant_id'")

# Q8: Column identifying food type
print("\nQ8 Proof - Food type column:")
print("Column -> 'cuisine'")

# Q9: User repetition for multiple orders
print("\nQ9 Proof - User repetition:")
sample_user = final_df["user_id"].value_counts().idxmax()
print("Sample user_id:", sample_user)
print("Number of orders by this user:",
      final_df[final_df["user_id"] == sample_user].shape[0])

Orders columns: ['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name']
Users columns: ['user_id', 'name', 'city', 'membership']
Restaurants columns: ['restaurant_id', 'restaurant_name', 'cuisine', 'rating']

Q1 Proof - Join key between orders and users:
Used key -> 'user_id'

Q2 Proof - Cuisine & rating source:
Source dataframe -> restaurants
Original format -> SQL (.sql)

Q3 Proof - Row count check:
Orders rows: 10000
Final merged rows: 10000

Q4 Proof - Missing user values:
Empty DataFrame
Columns: [order_id, user_id, restaurant_id, order_date, total_amount, restaurant_name_x, name, city, membership, restaurant_name_y, cuisine, rating]
Index: []

Q5 Proof - Pandas join function used:
Function -> pandas.merge()

Q6 Proof - Membership column origin:
membership column found in users.json

Q7 Proof - Join key with restaurants:
Used key -> 'restaurant_id'

Q8 Proof - Food type column:
Column -> 'cuisine'

Q9 Proof - User repetition:
Sample user_id: 2973
